In [ ]:
#we may need some code in the ../python directory and/or matplotlib styles
import sys
sys.path.append('../python/')

#matplotlib for plotting
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.style.use('../mplstyles/stylelib/standard.mplstyle')

#other computational libraries
import numpy as np
import pandas as pd
import scipy.constants as co
import scipy.stats as ss
import itertools
import pickle
from scipy import signal

In [ ]:
import ENDF6

In [ ]:
#get some cross section data
f = open('xn_data/n-014_Si_028.endf')
lines = f.readlines()
sec = ENDF6.find_section(lines, MF=4, MT=2)  # elastic angular distributions. 
#x, y = ENDF6.read_table(sec)

In [ ]:
for i in sec:
    #print(i)
    arr = np.char.split(i)
    #print(np.shape(arr))
    if(np.shape(arr)==()): continue
    #print([np.fromstring(x) for x in arr])

In [ ]:
print(ENDF6.list_content(sec))

In [ ]:
sec_el = ENDF6.find_section(lines, MF=3, MT=2)

f.close()

In [ ]:
Eel, sigel = ENDF6.read_table(sec_el)

In [ ]:
print(Eel)
print(sigel)

In [ ]:
from pylab import *
#plot quick
fig = plt.figure()
ax1 = fig.add_subplot(111)
plot(Eel, sigel)
ax1.set_yscale('log')
ax1.set_xscale('log')
xlabel('Neutron energy [eV]')
ylabel('Cross-section [barn]')
show()

In [ ]:
sec_res = ENDF6.find_section(lines, MF=2, MT=151)

In [ ]:
for i in sec_res:
    print(i)
    arr = np.char.split(i)
    #print(np.shape(arr))
    if(np.shape(arr)==()): continue
    #print([np.fromstring(x) for x in arr])

In [ ]:
import pandas as pds
si28_elastic = pds.read_csv('xn_data/si28_el.txt', skiprows=11,skipfooter=2, \
                         names=['neutE', 'xn'],sep='\s+',engine='python')

print (si28_elastic.head(100))

si29_elastic = pds.read_csv('xn_data/si29_el.txt', skiprows=11,skipfooter=2, \
                         names=['neutE', 'xn'],sep='\s+',engine='python')

si30_elastic = pds.read_csv('xn_data/si30_el.txt', skiprows=11,skipfooter=2, \
                         names=['neutE', 'xn'],sep='\s+',engine='python')

In [ ]:
si28_neute = np.asarray(si28_elastic["neutE"],dtype=float)
si28_xn = np.asarray(si28_elastic["xn"],dtype=float)

si29_neute = np.asarray(si29_elastic["neutE"],dtype=float)
si29_xn = np.asarray(si29_elastic["xn"],dtype=float)

si30_neute = np.asarray(si30_elastic["neutE"],dtype=float)
si30_xn = np.asarray(si30_elastic["xn"],dtype=float)

In [ ]:
d = diff(si28_neute)
d=np.append(d,0)
print(np.shape(d))
print(d[d<=0])
si28_neute = si28_neute[d>0]
si28_xn = si28_xn[d>0]

import scipy
si28f=scipy.interpolate.UnivariateSpline(
        si28_neute,
        si28_xn,
        k=3,
        s=0,
        check_finite=True)


In [ ]:
print(si28f([3,4]))

In [ ]:
#plot quick
fig = plt.figure()
ax1 = fig.add_subplot(111)
plot(si28_neute, si28_xn)
plot(si29_neute, si29_xn)
plot(si30_neute, si30_xn)
plot(si28_neute,si28f(si28_neute))
ax1.set_yscale('log')
ax1.set_xscale('log')
xlabel('Neutron energy [MeV]')
ylabel('Cross-section [barn]')
show()

In [ ]:
sec = ENDF6.find_section(lines, MF=4, MT=2)
for i in sec:
    #print(i)
    arr = np.char.split(i)
    #print(np.shape(arr))
    if(np.shape(arr)==()): continue
    #print([np.fromstring(x) for x in arr])

In [ ]:
arr=np.str_.split(sec[3])
num=int(arr[0])
print(num)
al = np.zeros((num,12))

In [ ]:
print(np.shape(al))

In [ ]:
readl=True
linecnt=0
ecnt=0
tote=num
mpoles=0
for ln in sec[4:-1]:
    print(ln)
    #break away if you're done reading e points
    if ecnt==tote:
      break
    #read the number of multipoles
    if readl:
      arr=np.str_.split(ln)
      mpoles=int(arr[4])
      print(mpoles)
      readl=False
    else:
      #arr=np.str_.split(ln)
      a = ENDF6.read_line(ln)
      print(a)
      i1=linecnt*6
      i2=i1+6
      print(i1,i2)
      al[ecnt,i1:i2] = a
      #ecnt+=1
      linecnt+=1
      if linecnt==np.ceil(mpoles/6.0):
         linecnt=0
         ecnt+=1
         readl=True
        
    arr = np.char.split(ln)
    #print(np.shape(arr))
    if(np.shape(arr)==()): continue
    #print([np.fromstring(x) for x in arr])

In [ ]:
print(al)

In [ ]:
print(al[412,:])

In [ ]:
print(al[3,:])

In [ ]:
import re

In [ ]:
str1='-5.634-5'
str2='4.32-4'



In [ ]:
x = re.sub('[0-9]-[0-9]','5e-5',str1)
print(x)

In [ ]:
x = re.sub('[0-9]-[0-9]','5e-5',str2)
print(x)

In [ ]:
print(str1.replace("-", "e-").lstrip("e"))

In [ ]:
num = float(re.sub(r"([0-9\-+.][0-9.]*)([\-+])(\d+)", r"\1e\2\3", str1))
print(num)

In [ ]:
print(str2.replace("-", "e-").lstrip("e"))

In [ ]:
num = float(re.sub(r"([0-9\-+.][0-9.]*)([\-+])(\d+)", r"\1e\2\3", str2))
print(num)

In [ ]:
print(str1.replace("-", "e-").replace("+", "e+").lstrip("e"))

In [ ]:
import ENDF6el as endfel

f = endfel.fetch_elastic(filename='xn_data/si28_el.txt')
print(f([3,4]))

In [ ]:
(en,al)=endfel.fetch_elastic_angular('xn_data/n-014_Si_028.endf')
print(np.shape(en))
print(np.shape(al))

In [ ]:
print(al)

In [ ]:
print(en)

In [ ]:
l = np.arange(0,37)
print(l)
prel = 2*l+1/2
print(prel)

In [ ]:
coeff = np.append([1],al[1,:])
print(coeff*prel)
adist = np.polynomial.legendre.Legendre(coeff*prel)

coeff1 = np.append([1],al[50,:])
print(coeff1*prel)
adist1 = np.polynomial.legendre.Legendre(coeff1*prel)

coeff2 = np.append([1],al[100,:])
print(coeff2*prel)
adist2 = np.polynomial.legendre.Legendre(coeff2*prel)

coeff3 = np.append([1],al[150,:])
print(coeff3*prel)
adist3 = np.polynomial.legendre.Legendre(coeff3*prel)

In [ ]:
print(adist(1.0))

In [ ]:
adistv = np.vectorize(adist)
adist1v = np.vectorize(adist1)
adist2v = np.vectorize(adist2)
adist3v = np.vectorize(adist3)

In [ ]:
#plot quick
ct = np.linspace(-1.0,1.0,100)
fig = plt.figure()
ax1 = fig.add_subplot(111)
plot(ct, adistv(ct))
plot(ct, adist1v(ct))
plot(ct, adist2v(ct))
plot(ct, adist3v(ct))
#ax1.set_yscale('log')
#ax1.set_xscale('log')
ax1.set_ylim(0.0,2)
xlabel('cos(thet)')
ylabel('ang dist [/cos(th)]')
show()